In [ ]:
# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install scipy
# !pip install seaborn

In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import rc
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import re
import glob
import seaborn as sns
from datetime import datetime, timedelta

In [2]:
os.getcwd()

'/Users/b05/Desktop/YEAR_DREAM/FC_PJT'

In [3]:
# Checking Fonts

rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
font_path = fm.findfont('AppleGothic')
if not font_path:
    print('Warning: AppleGothic font not found')
else:
    print("AppleGothic font found at ", font_path)

font_prop = fm.FontProperties(fname=font_path, size=12)

AppleGothic font found at  /System/Library/Fonts/Supplemental/AppleGothic.ttf


In [4]:
# Creating variables for data paths
dataset = "./dataset/"
GROUPED = dataset + "Grouped/"

In [5]:
if not os.path.exists(dataset + "Grouped"):
    os.mkdir(dataset + "Grouped")

In [6]:
file_locations = {
    'grouped': GROUPED    
}

all_file_locations = {}
for key, value in file_locations.items():
    all_file_locations[key] = glob.glob(value + "*.csv")

In [7]:
df = pd.read_csv(dataset+'eda-proj-fc-purchase.csv')

In [8]:
df.rename(columns={"거래id":"transaction_id", "유형":"l_type", "고객id":"customer_id","코스ID":"courseID","사이트":"site", "포맷":"l_format", "카테고리":"l_categories", "코스(상품) 이름":"course_name", "거래일자":"transaction_date_time","쿠폰이름":"coupon_name","판매가격":"sold_price","결제수단":"payment_method","실거래금액":"actual_sold_price","쿠폰할인액":"coupon_discount","거래금액":"transaction_amount","환불금액":"refund_amount"}, inplace=True)
df.to_csv(dataset + "rowdata_eng.csv", index=False)

In [9]:
rowdata_eng = pd.read_csv(dataset+"rowdata_eng.csv")

In [10]:
df = rowdata_eng['transaction_date_time'].str.split(" ", n=5, expand=True)
rowdata_eng['Date'] = df[0].str.cat([df[1], df[2]], sep="")
rowdata_eng['Time'] = df[4].str.cat([df[3]], sep=" ")
rowdata_eng.to_csv(dataset + "times.csv", index=False)

In [12]:
times = pd.read_csv(dataset + "times.csv")

# for i in times["Time"]:
#     print(i)
    

0         10:12:33 오후
1          6:12:57 오후
2         11:12:15 오후
3         11:12:03 오후
4         11:12:10 오후
             ...     
159323    12:01:08 오전
159324    12:01:12 오전
159325    12:01:21 오전
159326    11:01:24 오전
159327     4:01:00 오전
Name: Time, Length: 159328, dtype: object

In [ ]:
# for i, time in times.iteritems():
for i in rowdata_eng["Time"]:
    

# if 오후" in time:
#     df[3] = "PM"
# else:
#     df[3] = "AM"

# row_data_dtsplit = rowdata_eng

# rowdata_eng.to_csv(dataset + "times.csv", index=False)

In [ ]:
tsplit = row_data_dtsplit["Time"]
for i, time in tsplit.iteritems():
    if "오후" in time:
        time = time.replace("오후 ","")
        print(time)
        if int(time.split(":")[0]) == "12" and "오후" in time:
            time = time.replace("오후 ","")
            
        else:
            x = int(time.split(":")[0]) + 12
            y = time.split(":")[1]
            z = time.split(":")[2]
            xyz = [str(x),":", str(y), ":", str(z)]
            time_str = "".join(xyz)
            row_data_dtsplit.at[i, "Time"] = time_str
    else:
        time = time.replace("오전 ", "")
        if int(time.split(":")[0]) == 12:
            x = 0
            time.split(":")[0] = "0"
            y = time.split(":")[1]
            z = time.split(":")[2]
            xyz = [str(x),":", str(y), ":", str(z)]
            time_str = "".join(xyz)
            row_data_dtsplit.at[i, "Time"] = time_str
        else:
            time = time.replace("오전 ", "")
    

row_time_data = row_data_dtsplit
row_data_dtsplit.to_csv(dataset + "row_data_dtsplit.csv", index=False)

In [ ]:
def clean_course_data(course_name):
    if course_name.startswith("("):
        if "~" in course_name:
            return course_name.split(")")[1]
        else:
            return course_name
    else:
        return course_name
    
row_time_data["course_name"] = row_time_data["course_name"].apply(clean_course_data)
course_name_cleansed = row_time_data
# row_time_data.to_csv(dataset +"course_name_cleansed.csv", index = False)

In [ ]:
def group_columns(row):
    parts = re.split("[:|-]", row["course_name".lower()])
    if len(parts) <= 1:
        if parts[0].startswith("[무료 강의]"):
            return "[무료 강의]"
        return ("Other")
#-----------------------------------------------------------------------------
    elif " " in parts[0]:
        parts[0] = parts[0].replace(" ","")
        if parts[0].startswith("[Kit]"):
            return "Kit"
        elif parts[0].startswith("(("):
            return parts[0].split("))")[1]
        elif parts[0].startswith("("):
            return parts[0].split(")")[1]
        elif parts[0].startswith("["):
            return parts[0].split("]")[1]
        else:
            return parts[0]

course_name_cleansed["course_group"] = course_name_cleansed.apply(group_columns, axis=1)
grouped_df = course_name_cleansed.groupby("course_group")
for group_name, group_df in grouped_df:
    print("Group: ", group_name)
    groupname = group_name.replace("/","_")
    filename = f"{groupname.strip()}.csv"
    group_df.to_csv(GROUPED + f"{filename}", index=False)
    

In [ ]:
for file in all_file_locations["grouped"]:
    df = pd.read_csv(file)
    print(file, file.split("/")[-1].split(".")[0] ,len(set(df["course_name"])))

In [ ]:
# df_new = pd.DataFrame()
df_new = pd.read_csv(GROUPED + "Other.csv")
for file in all_file_locations["grouped"]:
    if file == GROUPED + "Other.csv":
        continue
    else:
        df = pd.read_csv(file)
        if len(set(df["course_name"])) == 1:
            df_new = pd.concat([df_new, df])
        else:
            pass

pd.concat([df_new, df_other])
df_new.to_csv(GROUPED + "Others.csv", index=False)